In [ ]:
import numpy as np
import cv2
import os
import skimage
from skimage.io import imread, imshow
import json
from matplotlib import pyplot as plt
from PIL import Image, ImageEnhance

import cv2

def pre_processImg(img):
    
    #-----Reading the image-----------------------------------------------------
    #img = cv2.imread(r'E:\Leuk study re-designed\C-NMC\Low imbalance\Train - 1 to 10 ratio\hem\UID_H1_1_1_hem.bmp', 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #cv2.imshow("img",img) 

    #-----Converting image to LAB Color model----------------------------------- 
    lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    #cv2.imshow("lab",lab)

    #-----Splitting the LAB image to different channels-------------------------
    l, a, b = cv2.split(lab)
    #cv2.imshow('l_channel', l)
    #cv2.imshow('a_channel', a)
    #cv2.imshow('b_channel', b)

    #-----Applying CLAHE to L-channel-------------------------------------------
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    #cv2.imshow('CLAHE output', cl)

    #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
    limg = cv2.merge((cl,a,b))
    #cv2.imshow('limg', limg)

    #-----Converting image from LAB Color model to RGB model--------------------
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    print('Min max balues: ', np.max(final), np.min(final), final.dtype)
    im = Image.fromarray(final)
    enhancer = ImageEnhance.Brightness(im)

    factor = 2.0 #gives original image
    im_output = enhancer.enhance(factor)
    #im_output.save('enhanced-2.0.0.png')
    #cv2.imshow('final', final)

    #_____END_____#

    #frame = increase_brightness(final, value=20)

    #fig = plt.figure(figsize=(10, 10))

    # ax1 = fig.add_subplot(2, 2, 1)
    # ax1.axis("off")
    # ax1.title.set_text('Original')
    # ax2 = fig.add_subplot(2, 2, 2)
    # ax2.axis("off")
    # ax2.title.set_text("Equalized")
    # ax3 = fig.add_subplot(2, 2, 3)
    # ax3.axis("off")
    # ax3.title.set_text("Enhanced")

    # ax1.imshow(img, cmap='gray')
    # ax2.imshow(final, cmap='gray')
    # ax3.imshow(frame, cmap='gray')

    #cv2.imwrite('./1.png', final)
    return im_output #PIL -Image returning


path_src = r'F:\Leuk study re-designed\C-NMC\High imbalance\Test\hem'
path_tgt = r'F:\Leuk study re-designed\C-NMC\High imbalance\Test\enhanched\hem'

all_list = os.listdir(path_src)
for x in range(len(all_list)):
    img = cv2.imread(os.path.join(path_src, all_list[x]))
    img = pre_processImg(img)
    img.save(os.path.join(path_tgt, all_list[x]))
print('Success')

In [18]:
from PIL import Image, ImageEnhance

#read the image
im = Image.open("1.png")

#image brightness enhancer
enhancer = ImageEnhance.Brightness(im)

factor = 2.0 #gives original image
im_output = enhancer.enhance(factor)
im_output.save('enhanced-2.0.png')



In [19]:
x_train = np.load("x_train_leuknet.npy")
y_train = np.load("y_train_leuknet.npy")

In [20]:
x_train.shape, y_train.shape

((3727, 210, 210, 3), (3727,))

In [22]:
from keras.utils import np_utils
num_classes = 2

y_train = np_utils.to_categorical(y_train,num_classes)

In [23]:
x_train.shape, y_train.shape

((3727, 210, 210, 3), (3727, 2))

In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(np.argmax(y_train, axis=1)),np.argmax(y_train, axis=1))
print("class weights: ",class_weights)